In [ ]:
import warnings
warnings.filterwarnings('ignore')

import re
import os
import string
import datetime as dt
import shutil
import json
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, Dataset

import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing import text

device = torch.device('cuda') if torch.cuda.is_available else 'cpu'
import datasets

from collections import Counter

In [ ]:
# Cosine Simularity
class Similarity(nn.Module):
    """
    Dot product or cosine similarity
    """

    def __init__(self, temp = 0.05):
        super().__init__()
        self.temp = temp
        self.cos = nn.CosineSimilarity(dim=-1)

    def forward(self, x, y):
        return self.cos(x, y) / self.temp

# Backbone Model
class Custom_tf_encoder(nn.Module):
    def __init__(self, used_word, embed_dim, nhead, encoder_layers, dropout=0.1, max_len=500):
        super().__init__()
        self.embedding_layer = nn.Embedding(used_word, embed_dim)
        self.encoders = nn.ModuleList([nn.TransformerEncoderLayer(
            d_model=embed_dim, 
            nhead=nhead, 
            dim_feedforward=(embed_dim*2), 
            dropout=dropout, 
            activation='gelu', 
            batch_first=True) for _ in range(encoder_layers)])
#         self.avgpool = nn.AvgPool1d(kernel_size=embed_dim)
#         self.fc = nn.Linear(embed_dim, 2)
#         self.maxpool = nn.MaxPool2d(kernel_size = (max_len,1))
    
    def mk_padding_mask(self, text):
        # <pad>: 0
        return torch.eq(text, 0)
        
    def forward(self, text):
        x = self.embedding_layer(text)
        padding_mask = self.mk_padding_mask(text).to(x.device)
        for layer in self.encoders:
            x = layer(x, src_key_padding_mask=padding_mask)
#         x = self.maxpool(x) # (batch, 1, embed_dim)
#         x = x.squeeze(1) # (batch, embed_dim)
#         x = self.fc(x)
        
        return x

In [ ]:
# SIMCES Contrastive Learning Model
class transformer_cl(nn.Module):
    def __init__(self, backbone):
        super().__init__()
        self.backbone = backbone
        self.Similarity = Similarity()
        
    def forward(self, x):
        emb_a = self.backbone(x)
        emb_b = self.backbone(x)
        emb_a, emb_b = emb_a[:, -1], emb_b[:, -1]
        cos_sim = self.Similarity(emb_a.unsqueeze(1), emb_b.unsqueeze(0))
        return cos_sim
        